In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool, AgentExecutor, AgentType

# Initialize the model 'Conversational' variant of GPT-4o
llm = ChatOpenAI(temperature=0.9, model_name="gpt-4o")

# Initialize SerpAPI tool using wrapper
serpapi_wrapper = SerpAPIWrapper()

serpapi_tool = Tool(
    name="serpapi",
    description="busca informacion en la web usando una tool -serpapi-.",
    func=serpapi_wrapper.run
)

# Create an agent with SerpAPI tool
agent = initialize_agent(
    tools=[serpapi_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True
)

# First chain: Agent generates text using SerpAPI
story_prompt = PromptTemplate.from_template(
    "Usa la herramienta de busqueda para buscar un texto de Borges en la web (de 3 a 5 lineas) relacionadas con alguno de sus mas famosos textos."
)
# story_chain = story_prompt | agent | StrOutputParser()
story_chain = story_prompt | agent | (lambda x: x['output']) | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template(
    "Analiza en sentimiento de la historia:\n{story}"
)
analysis_chain = analysis_prompt | llm | StrOutputParser()

# Third chain: Rewrite the story with a specified sentiment
rewrite_prompt = PromptTemplate.from_template(
    "Reescribe la siguiente historia con un sentimiento {sentiment}:\n{story}"
)
rewrite_chain = rewrite_prompt | llm | StrOutputParser()

# Combine chains
# story_with_analysis = story_chain | analysis_chain
# Run the combined chain
# story_analysis = story_with_analysis.invoke({"topic": "a rainy day"})
# print("\nAnalysis:", story_analysis)

# Using RunnablePassthrough.assign to preserve original data
enhanced_chain = RunnablePassthrough.assign(
    story=story_chain  # Add 'story' key with generated content
).assign(
    analysis=analysis_chain  # Add 'analysis' key with analysis of the story
).assign(
    rewritten_story=lambda x: rewrite_chain.invoke({"story": x["story"], "sentiment": "humoristico"})
)

# Execute the chain
result = enhanced_chain.invoke({"topic": "El azar"})
print("Story:", result['story'])
print("\nAnalysis:", result['analysis'])
print("\nRewritten Story (Happy):", result['rewritten_story'])
print(result.keys())



/tmp/ipykernel_4372/1573371254.py:21: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


Story: "El jardín de los senderos que se bifurcan. Algo se agitó en mi recuerdo y pronuncié con incomprensible seguridad: —El jardín de mi antepasado Ts'ui Pên."

Analysis: La historia mencionada, "El jardín de los senderos que se bifurcan," es un cuento del escritor argentino Jorge Luis Borges. Al analizar el sentimiento de este fragmento en particular, es importante considerar el contexto general de la obra y los temas que Borges suele explorar.

En esta línea, el sentimiento predominante podría ser uno de curiosidad e intriga. La mención de "algo se agitó en mi recuerdo" sugiere una conexión personal y profunda con el tema, mientras que la referencia al "jardín de mi antepasado Ts'ui Pên" evoca un sentido de misterio y exploración del pasado. Este tipo de conexión ancestral y la seguridad inexplicable con que el protagonista pronuncia estas palabras pueden resonar con sentimientos de nostalgia, búsqueda de identidad, y una fascinación por los enigmas.

Además, la obra en su totalida